<a href="https://colab.research.google.com/github/dolmani38/AMD-GAN/blob/main/torch_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [2]:


from __future__ import print_function
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils


In [3]:
cudnn.benchmark = True

#set manual seed to a constant get a consistent output
manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

#loading the dataset
dataset = dset.CIFAR10(root="./data", download=True, train=True,
                           transform=transforms.Compose([
                               #transforms.Resize(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=128,
                                         shuffle=True, num_workers=2)

#checking the availability of cuda devices
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Random Seed:  6145



Extracting ./data/cifar-10-python.tar.gz to ./data


In [4]:
class IgnoreLabelDataset(torch.utils.data.Dataset):
    def __init__(self, orig):
        self.orig = orig

    def __getitem__(self, index):
        return self.orig[index][0]

    def __len__(self):
        return len(self.orig)

ld = IgnoreLabelDataset(dataset)
print(len(ld))
print(ld[0].shape)

50000
torch.Size([3, 32, 32])


In [5]:
nc=3

# number of gpu's available
ngpu = 1
# input noise dimension
nz = 100
# number of generator filters
ngf = 64
#number of discriminator filters
ndf = 64


In [6]:

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
                        
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
            return output

netG = Generator(ngpu).to(device)
netG.apply(weights_init)
#load weights to test the model
#netG.load_state_dict(torch.load('weights/netG_epoch_24.pth'))
print(netG)


Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): Tanh()
  )
)


In [7]:

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            #nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            #nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(nc, ndf * 2, 4, 2, 1, bias=False),
            #nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

netD = Discriminator(ngpu).to(device)
netD.apply(weights_init)
#load weights to test the model 
#netD.load_state_dict(torch.load('weights/netD_epoch_24.pth'))
print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)


In [8]:

criterion = nn.BCELoss()

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

fixed_noise = torch.randn(128, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

niter = 25
g_loss = []
d_loss = []

In [9]:

for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        rlabel = torch.full((batch_size,), real_label, dtype=torch.float32, device=device)
        flabel = torch.full((batch_size,), fake_label, dtype=torch.float32, device=device)

        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)

        errD = criterion(netD(real_cpu), rlabel) + criterion(netD(fake.detach()), flabel)
        errD.backward()
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()

        errG = criterion(netD(fake), rlabel)
        errG.backward()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f ' % (epoch, niter, i, len(dataloader), errD.item(), errG.item()))
        
        #save the output
        if i % 100 == 0:
            print('saving the output')
            vutils.save_image(real_cpu,'./drive/MyDrive/t_DCGAN/output/real_samples.png',normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),'./drive/MyDrive/t_DCGAN/output/fake_samples_epoch_%03d.png' % (epoch),normalize=True)
    
    # Check pointing for every epoch
    torch.save(netG.state_dict(), './drive/MyDrive/t_DCGAN/weights/netG_epoch_%d.pth' % (epoch))
    torch.save(netD.state_dict(), './drive/MyDrive/t_DCGAN/weights/netD_epoch_%d.pth' % (epoch))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[12/25][133/391] Loss_D: 0.2569 Loss_G: 2.8886 
[12/25][134/391] Loss_D: 0.2436 Loss_G: 2.8783 
[12/25][135/391] Loss_D: 0.3297 Loss_G: 2.8382 
[12/25][136/391] Loss_D: 0.3373 Loss_G: 2.3662 
[12/25][137/391] Loss_D: 0.4544 Loss_G: 3.5773 
[12/25][138/391] Loss_D: 0.6984 Loss_G: 1.3458 
[12/25][139/391] Loss_D: 0.4733 Loss_G: 3.3451 
[12/25][140/391] Loss_D: 0.3019 Loss_G: 3.3028 
[12/25][141/391] Loss_D: 0.2932 Loss_G: 2.7603 
[12/25][142/391] Loss_D: 0.3355 Loss_G: 2.8879 
[12/25][143/391] Loss_D: 0.3733 Loss_G: 2.8200 
[12/25][144/391] Loss_D: 0.4129 Loss_G: 1.9700 
[12/25][145/391] Loss_D: 0.4087 Loss_G: 3.7173 
[12/25][146/391] Loss_D: 0.3036 Loss_G: 3.0100 
[12/25][147/391] Loss_D: 0.3060 Loss_G: 2.2205 
[12/25][148/391] Loss_D: 0.5184 Loss_G: 3.7026 
[12/25][149/391] Loss_D: 0.5328 Loss_G: 1.9803 
[12/25][150/391] Loss_D: 0.3997 Loss_G: 2.8787 
[12/25][151/391] Loss_D: 0.2740 Loss_G: 3.5979 
[12/25][152/391] Loss_D: 0.3780 Loss_G: 2.2734 
[12/

In [10]:
!git clone https://github.com/sbarratt/inception-score-pytorch.git

Cloning into 'inception-score-pytorch'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 43 (delta 0), reused 1 (delta 0), pack-reused 40
Unpacking objects: 100% (43/43), done.


In [11]:
import sys
sys.path.append('./inception-score-pytorch')
from inception_score import inception_score

In [17]:

print(inception_score(ld, cuda=True, batch_size=32, resize=True, splits=10))

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
./inception-score-pytorch/inception_score.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


(9.672786661200819, 0.1499139460804442)


In [12]:
eval_images = []
for i in range(600):
    noise = torch.randn(batch_size, nz, 1, 1, device=device)
    with torch.no_grad():
        fake = netG(noise).detach().cpu()
    eval_images += fake

print(len(eval_images))
print(eval_images[0].shape)
print(inception_score(eval_images, cuda=True, batch_size=32, resize=True, splits=10))

48000
torch.Size([3, 32, 32])


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
./inception-score-pytorch/inception_score.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


(4.673854281750242, 0.02680361664162138)


Dual Discriminator

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [25]:
netD1 = Discriminator(ngpu).to(device)
netD1.apply(weights_init)
print(netD1)

netD2 = Discriminator(ngpu).to(device)
netD2.apply(weights_init)
print(netD2)

netG0 = Generator(ngpu).to(device)
netG0.apply(weights_init)
#load weights to test the model
#netG.load_state_dict(torch.load('weights/netG_epoch_24.pth'))
print(netG0)


Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)
Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=Fals

In [26]:

criterion = nn.BCELoss()

# setup optimizer
optimizerD1 = optim.Adam(netD1.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerD2 = optim.Adam(netD2.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG0 = optim.Adam(netG0.parameters(), lr=0.0002, betas=(0.5, 0.999))

fixed_noise = torch.randn(128, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

niter = 25
g_loss = []
d_loss = []

In [28]:

for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        rlabel = torch.full((batch_size,), real_label, dtype=torch.float32, device=device)
        flabel = torch.full((batch_size,), fake_label, dtype=torch.float32, device=device)
        
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG0(noise)

        netD1.zero_grad()
        errD1 = criterion(netD1(real_cpu), rlabel) + criterion(netD1(fake.detach()), flabel)
        errD1.backward()
        optimizerD1.step()

        netD2.zero_grad()
        errD2 = criterion(netD2(real_cpu), rlabel) + criterion(netD2(fake.detach()), flabel)
        errD2.backward()
        optimizerD2.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG0.zero_grad()

        errG0 = criterion(netD1(fake), rlabel) + criterion(netD2(fake), rlabel)
        errG0.backward()
        optimizerG0.step()


        print('[%d/%d][%d/%d] Loss_D1: %.4f Loss_D2: %.4f Loss_G: %.4f' % (epoch, niter, i, len(dataloader), errD1.item(),errD2.item(), errG0.item()))
        
        #save the output
        if i % 100 == 0:
            print('saving the output')
            vutils.save_image(real_cpu,'./drive/MyDrive/t_DCGAN/output/2d_real_samples.png',normalize=True)
            fake = netG0(fixed_noise)
            vutils.save_image(fake.detach(),'./drive/MyDrive/t_DCGAN/output/2d_fake_samples_epoch_%03d.png' % (epoch),normalize=True)
    

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[12/25][133/391] Loss_D1: 0.3677 Loss_D2: 0.6569 Loss_G: 4.5343
[12/25][134/391] Loss_D1: 0.3502 Loss_D2: 0.6002 Loss_G: 5.2384
[12/25][135/391] Loss_D1: 0.3567 Loss_D2: 0.4381 Loss_G: 5.7672
[12/25][136/391] Loss_D1: 0.4205 Loss_D2: 0.5317 Loss_G: 4.4359
[12/25][137/391] Loss_D1: 0.4773 Loss_D2: 0.5600 Loss_G: 6.6047
[12/25][138/391] Loss_D1: 0.4481 Loss_D2: 0.5508 Loss_G: 3.5753
[12/25][139/391] Loss_D1: 0.4925 Loss_D2: 0.4725 Loss_G: 6.3655
[12/25][140/391] Loss_D1: 0.7016 Loss_D2: 0.5940 Loss_G: 3.7486
[12/25][141/391] Loss_D1: 0.9029 Loss_D2: 0.4382 Loss_G: 6.8877
[12/25][142/391] Loss_D1: 0.7239 Loss_D2: 0.5388 Loss_G: 4.2192
[12/25][143/391] Loss_D1: 0.2769 Loss_D2: 0.4204 Loss_G: 6.3268
[12/25][144/391] Loss_D1: 0.5037 Loss_D2: 0.6177 Loss_G: 4.4501
[12/25][145/391] Loss_D1: 0.3865 Loss_D2: 0.4743 Loss_G: 6.2276
[12/25][146/391] Loss_D1: 0.5777 Loss_D2: 0.4448 Loss_G: 4.6649
[12/25][147/391] Loss_D1: 0.5215 Loss_D2: 0.4997 Loss_G: 5.3438
[12/

In [29]:
eval_images = []
for i in range(600):
    noise = torch.randn(batch_size, nz, 1, 1, device=device)
    with torch.no_grad():
        fake = netG0(noise).detach().cpu()
    eval_images += fake

print(len(eval_images))
print(eval_images[0].shape)
print(inception_score(eval_images, cuda=True, batch_size=32, resize=True, splits=10))

48000
torch.Size([3, 32, 32])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
./inception-score-pytorch/inception_score.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


(4.7815574018604, 0.052156231289132625)
